<a href="https://colab.research.google.com/github/RSOmi05/TinyLlama_v_1.1-for-Email-Subject-Lines-Generation/blob/main/Source_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf ~/.cache/huggingface
!rm -rf ~/.cache/torch
!nvidia-smi  # Check GPU memory usage

Fri Jun 13 11:57:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
!pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu124
!pip install nvidia-nccl-cu12

Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 108.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidi

In [3]:
import os
os.environ["XLA_FLAGS"] = "--xla_gpu_cuda_data_dir=/usr/local/cuda"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
import torch
torch.cuda.init()
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

CUDA Available: True
CUDA Device: NVIDIA A100-SXM4-40GB


In [4]:
# Install packages with version constraints
!pip install -q triton==3.2.0 transformers==4.51.3 trl peft==0.11.1 datasets==3.4.1 accelerate==0.34.1 bitsandbytes==0.39.0 evaluate==0.4.3
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install "git+https://github.com/unslothai/unsloth-zoo.git"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 132.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.3/324.3 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-h3ygtkyh/unsloth_

  Cloning https://github.com/unslothai/unsloth-zoo.git to /tmp/pip-req-build-e5y5jplj
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth-zoo.git /tmp/pip-req-build-e5y5jplj
  Resolved https://github.com/unslothai/unsloth-zoo.git to commit f22242b47f67c9ad7aab3450563ab58b85784772
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:
# Importing other libraries
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, AutoPeftModelForCausalLM
from datasets import load_dataset

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
# Enable 4-bit quantization to lower memory usage
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Loading model with 4-bit quantization
target_model = "TinyLlama/TinyLlama_v1.1"
tokenizer = AutoTokenizer.from_pretrained(target_model)
model = AutoModelForCausalLM.from_pretrained(target_model,
                                             quantization_config=quantization_config,
                                             device_map = "auto"
                                             )

# Set use_cache to False for compatibility with gradient checkpointing during training
model.config.use_cache = False

# THEN apply LoRA
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # Critical for Llama models!
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()  # Should show ~0.1% trainable params

for name, param in model.named_parameters():
    if "lora" in name:  # Ensure LoRA layers are trainable
        param.requires_grad = True

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [9]:
import numpy as np

target_model = "TinyLlama/TinyLlama_v1.1"
tokenizer = AutoTokenizer.from_pretrained(target_model)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

dataset = load_dataset("SAGI-1/subject_lines")

def tokenize_function(examples):
    prompt = [f"{body}\nSubject:" for body in examples["text"]]
    targets = examples["subject_lines"]
    full_texts = [p + " " + t for p, t in zip(prompt, targets)]

    tokenized = tokenizer(
        full_texts,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="np",
    )

    labels = np.array(tokenized["input_ids"])
    for i, (p, t) in enumerate(zip(prompt, targets)):
        prompt_ids = tokenizer(p, add_special_tokens=False)["input_ids"]
        labels[i, :len(prompt_ids)] = -100  # mask prompt

    tokenized["labels"] = labels
    return tokenized

tokenized_datasets = dataset.map(tokenize_function, batched=True)
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(500))
eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(50))

train-00000-of-00001.parquet:   0%|          | 0.00/10.3M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/565k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/59489 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3132 [00:00<?, ? examples/s]

Map:   0%|          | 0/59489 [00:00<?, ? examples/s]

Map:   0%|          | 0/3132 [00:00<?, ? examples/s]

In [14]:
import evaluate
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import numpy as np
import torch # Import torch to check for torch tensors

!pip install rouge_score

# Load ROUGE metric for evaluation
rouge_metric = evaluate.load("rouge")

# Compute metrics function for Trainer
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # For generative models, predictions are often logits.
    # We need to get the predicted token IDs by taking the argmax over the last dimension (logits).
    if isinstance(predictions, tuple):
        predictions = predictions[0] # Get the logits array

    # Ensure predictions is a numpy array or torch tensor of logits
    if isinstance(predictions, np.ndarray):
        # Take argmax along the token dimension to get predicted token IDs
        predicted_token_ids = np.argmax(predictions, axis=-1)
        predicted_token_ids = predicted_token_ids.tolist() # Convert to list of lists
    elif isinstance(predictions, torch.Tensor):
        # Take argmax along the token dimension to get predicted token IDs
        predicted_token_ids = torch.argmax(predictions, dim=-1)
        predicted_token_ids = predicted_token_ids.tolist() # Convert to list of lists
    else:
        # If it's neither, assume it's already a list or similar structure.
        # Print debug info and raise an error if it's not the expected format.
        print(f"Unexpected type for predictions: {type(predictions)}")
        print("Predictions structure:")
        print(predictions)
        raise TypeError("predictions should be a numpy array, torch tensor, or a tuple containing one.")


    # Decode generated tokens (predicted token IDs) to strings
    # This should now receive a List[List[int]]
    try:
        decoded_preds = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    except TypeError as e:
        print(f"Error during batch_decode with predicted_token_ids: {e}")
        print("predicted_token_ids structure before batch_decode causing error:")
        print(predicted_token_ids) # Print the problematic input
        raise # Re-raise the exception after printing


    # Replace -100 in labels as padding token id for decoding
    # Convert labels to numpy if it's not already (Trainer typically provides numpy or torch tensors)
    if isinstance(labels, torch.Tensor):
         labels = labels.cpu().numpy()

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Ensure labels is a list of lists for batch_decode
    if isinstance(labels, np.ndarray):
        labels = labels.tolist()
    elif isinstance(labels, torch.Tensor): # Should be handled by the above conversion
        labels = labels.tolist()
    else:
         print(f"Unexpected type for labels: {type(labels)}")
         print("Labels structure:")
         print(labels)
         raise TypeError("labels should be a numpy array or torch tensor.")


    # Decode labels to strings
    try:
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    except TypeError as e:
         print(f"Error during batch_decode with labels: {e}")
         print("labels structure before batch_decode causing error:")
         print(labels) # Print the problematic input
         raise # Re-raise the exception after printing


    # Compute ROUGE scores
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Format results as percentages
    result = {key: value * 100 for key, value in result.items()}
    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"],
    }

In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    label_names=["labels"],
    output_dir="./TinyLlama_Email_Subject_Lines_Generator",
    eval_strategy="epoch",  # Evaluation at epoch end
    save_strategy="epoch",  # Save checkpoints at epoch end to match eval
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    gradient_accumulation_steps=16,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=1000,
    save_steps=1000,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    greater_is_better=True,
    fp16=True,
    push_to_hub=False,
    report_to="none",
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant":False} # Recommended for newer PyTorch versions and when using `use_cache=True` which is often default.
)


In [16]:
from transformers import EarlyStoppingCallback
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stop if no improvement for 2 evals
)

# Train the model
# Added ignore_keys_for_eval to potentially save memory during evaluation
# This tells the trainer to not gather 'past_key_values' during evaluation, which can sometimes reduce memory usage for generative models.
trainer.train(ignore_keys_for_eval=["past_key_values"])

/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel
0,No log,5.719365,50.398462,48.739296,50.083581


TrainOutput(global_step=31, training_loss=7.380985383064516, metrics={'train_runtime': 244.7913, 'train_samples_per_second': 2.043, 'train_steps_per_second': 0.127, 'total_flos': 1578015242846208.0, 'train_loss': 7.380985383064516, 'epoch': 0.992})

In [17]:
# Evaluate
results = trainer.evaluate()
print(results)

# Define the directory to save your model and tokenizer
save_directory = "./TinyLlama_Email_Subject_Lines_Generator"

# Save the model
model.save_pretrained(save_directory)

# Save the tokenizer
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")

import json

# Evaluation results dictionary from trainer.evaluate()
eval_results = trainer.evaluate()

# Save to a JSON file
with open("Final_Evaluation_Results.json", "w") as f:
    json.dump(eval_results, f, indent=4)

print("Evaluation results saved to Final_Evaluation_Results.json")


model = AutoModelForCausalLM.from_pretrained(save_directory)
tokenizer = AutoTokenizer.from_pretrained(save_directory)


{'eval_loss': 5.719365119934082, 'eval_rouge1': 50.39846212460719, 'eval_rouge2': 48.73929625435906, 'eval_rougeL': 50.0835809807687, 'eval_runtime': 10.0947, 'eval_samples_per_second': 4.953, 'eval_steps_per_second': 4.953, 'epoch': 0.992}
Model and tokenizer saved to ./TinyLlama_Email_Subject_Lines_Generator
Evaluation results saved to Final_Evaluation_Results.json


In [22]:
# Generate predictions on the test set

test_dataset = eval_dataset.select(range(50))

# Use Trainer's predict method with generation enabled
predictions_output = trainer.predict(test_dataset)

# Get generated token IDs
# predictions_output.predictions should have shape (num_examples, sequence_length, vocab_size)
generated_logits = predictions_output.predictions

# Take argmax along the vocabulary dimension to get the predicted token IDs
# This will result in a shape of (num_examples, sequence_length)
generated_ids = np.argmax(generated_logits, axis=-1)

# Decode generated subject lines
# We need to iterate through each example's predicted token IDs and decode them
predicted_subject_lines = [tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
                          for ids in generated_ids]

# Getting original email bodies and true subject lines from the dataset
email_bodies = [ex["text"] for ex in test_dataset]
true_subject_lines = [ex["subject_lines"] for ex in test_dataset]

# Create a DataFrame with input, predicted output, and true output
import pandas as pd

df_test_output = pd.DataFrame({
    "email_body": email_bodies,
    "predicted_subject_line": predicted_subject_lines,
    "true_subject_line": true_subject_lines,
})

# Save to CSV (or JSON)
df_test_output.to_csv("Test_Output.csv", index=False)

print("Test output predictions saved to Test_Output.csv")

Test output predictions saved to Test_Output.csv


In [23]:
!git init
!git remote add origin https://github.com/RSOmi05/TinyLlama_v_1.1-for-Email-Subject-Lines-Generation.git

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [24]:
!git add ./TinyLlama_Email_Subject_Lines_Generator

In [25]:
!git commit -m "Add trained TinyLlama model for email subject line generation"

Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@3d853e7f01e4.(none)')
